In [56]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import pymongo
import datetime
from flask import jsonify
from bson.json_util import dumps, loads

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
client = Socrata("data.sfgov.org", None)
results = client.get("wg3w-h783",incident_year="2020", limit=100000)

In [4]:
results_df = pd.DataFrame.from_records(results)
results_df.head()

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,report_type_description,filed_online,incident_code,incident_category,incident_subcategory,incident_description,resolution,police_district,cad_number,intersection,cnn,analysis_neighborhood,supervisor_district,latitude,longitude,point
0,2020-02-05T20:00:00.000,2020-02-05T00:00:00.000,20:00,2020,Wednesday,2020-02-05T23:16:00.000,90044306374,900443,206028101,II,Coplogic Initial,True,06374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-29T11:30:00.000,2020-01-29T00:00:00.000,11:30,2020,Wednesday,2020-02-08T15:03:00.000,90055006372,900550,200097972,II,Initial,NaN,06372,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, $50-$200",Open or Active,Mission,200392271,18TH ST \ CASTRO ST,25813000,Castro/Upper Market,8,37.76088893209152,-122.4350007026991,"{'type': 'Point', 'coordinates': [-122.4350007..."
2,2020-02-08T13:25:00.000,2020-02-08T00:00:00.000,13:25,2020,Saturday,2020-02-08T13:28:00.000,90054312030,900543,200097803,II,Initial,NaN,12030,Weapons Offense,Weapons Offense,"Weapon, Deadly, Imitation or Laser Scope, Exhi...",Cite or Arrest Adult,Park,200391938,ASHBURY ST \ PIEDMONT ST,26216000,Haight Ashbury,8,37.765216184713424,-122.44596265042229,"{'type': 'Point', 'coordinates': [-122.4459626..."
3,2020-02-08T12:00:00.000,2020-02-08T00:00:00.000,12:00,2020,Saturday,2020-02-08T14:17:00.000,90053806244,900538,200097875,II,Initial,NaN,06244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Northern,200392099,MARINA BLVD \ LYON ST,33625000,Presidio,2,37.80472422408812,-122.44827914652447,"{'type': 'Point', 'coordinates': [-122.4482791..."
4,2020-02-08T22:32:00.000,2020-02-08T00:00:00.000,22:32,2020,Saturday,2020-02-08T22:32:00.000,90063407045,900634,190656599,VS,Vehicle Supplement,NaN,07045,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Truck",Open or Active,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
results_df["incident_category"].unique()

In [ ]:
# results_df["incident_category"].value_counts()

In [5]:
result_dropped_columns = results_df.drop(['incident_date','report_datetime','row_id','incident_id',
                                          'incident_number','report_type_code','report_type_description',
                                          'filed_online','incident_code','incident_subcategory','cad_number',
                                          'intersection','cnn','analysis_neighborhood','supervisor_district',
                                          'point'], axis=1)
# result_dropped_columns

In [6]:
result_dropped_columns = result_dropped_columns.dropna()

In [8]:
result_renamed = result_dropped_columns.rename(columns={'incident_datetime':'Datetime','incident_time':'Time',
                                         'incident_year':'Year','incident_day_of_week':'Week','incident_category':'Category',
                                         'incident_description':'Description','resolution':'Resolution',
                                         'police_district':'Police_Dist','latitude':'Lat','longitude':'Lon'})

In [9]:
result_renamed

,Datetime,Time,Year,Week,Category,Description,Resolution,Police_Dist,Lat,Lon
1,2020-01-29T11:30:00.000,11:30,2020,Wednesday,Larceny Theft,"Theft, Other Property, $50-$200",Open or Active,Mission,37.76088893209152,-122.4350007026991
2,2020-02-08T13:25:00.000,13:25,2020,Saturday,Weapons Offense,"Weapon, Deadly, Imitation or Laser Scope, Exhi...",Cite or Arrest Adult,Park,37.765216184713424,-122.44596265042229
3,2020-02-08T12:00:00.000,12:00,2020,Saturday,Larceny Theft,"Theft, From Locked Vehicle, >$950",Open or Active,Northern,37.80472422408812,-122.44827914652447
5,2020-02-06T06:15:00.000,06:15,2020,Thursday,Larceny Theft,"Theft, From Locked Vehicle, >$950",Open or Active,Bayview,37.74597422469076,-122.39704283791049
6,2020-02-07T07:30:00.000,07:30,2020,Friday,Burglary,"Burglary, Residence, Unlawful Entry",Open or Active,Mission,37.754627665339015,-122.43217749035517
...,...,...,...,...,...,...,...,...,...,...
13591,2020-01-28T09:22:00.000,09:22,2020,Tuesday,Larceny Theft,"Theft, Shoplifting, <$50",Open or Active,Taraval,37.74205567451305,-122.49818952795059
13593,2020-01-01T00:30:00.000,00:30,2020,Wednesday,Larceny Theft,"Theft, Other Property, $50-$200",Open or Active,Northern,37.788222699672026,-122.42021043633935
13594,2020-01-10T06:51:00.000,06:51,2020,Friday,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,Mission,37.7518282065593,-122.43855099872502
13595,2020-01-14T11:01:00.000,11:01,2020,Tuesday,Larceny Theft,"Theft, Shoplifting, $50-$200",Open or Active,Northern,37.77646430525107,-122.42626334607084


In [18]:
# result_renamed.dtypes

In [17]:
result_renamed.loc[result_renamed['Datetime'] > "2020-02-01" ]
# (result_dropped_columns['Week'] == 'Friday')
# q1['Week'].value_counts()

,Datetime,Time,Year,Week,Category,Description,Resolution,Police_Dist,Lat,Lon
2,2020-02-08T13:25:00.000,13:25,2020,Saturday,Weapons Offense,"Weapon, Deadly, Imitation or Laser Scope, Exhi...",Cite or Arrest Adult,Park,37.765216184713424,-122.44596265042229
3,2020-02-08T12:00:00.000,12:00,2020,Saturday,Larceny Theft,"Theft, From Locked Vehicle, >$950",Open or Active,Northern,37.80472422408812,-122.44827914652447
5,2020-02-06T06:15:00.000,06:15,2020,Thursday,Larceny Theft,"Theft, From Locked Vehicle, >$950",Open or Active,Bayview,37.74597422469076,-122.39704283791049
6,2020-02-07T07:30:00.000,07:30,2020,Friday,Burglary,"Burglary, Residence, Unlawful Entry",Open or Active,Mission,37.754627665339015,-122.43217749035517
7,2020-02-08T13:10:00.000,13:10,2020,Saturday,Warrant,"Warrant Arrest, Enroute To Outside Jurisdiction",Cite or Arrest Adult,Northern,37.78245048001555,-122.43929762619126
...,...,...,...,...,...,...,...,...,...,...
13511,2020-02-01T11:00:00.000,11:00,2020,Saturday,Larceny Theft,"Theft, From Locked Vehicle, $200-$950",Open or Active,Park,37.767448809535935,-122.46067684290128
13518,2020-02-01T21:30:00.000,21:30,2020,Saturday,Motor Vehicle Theft,"Vehicle, Stolen, Auto",Open or Active,Bayview,37.75724266322745,-122.39990737812333
13534,2020-02-02T11:30:00.000,11:30,2020,Sunday,Miscellaneous Investigation,Miscellaneous Investigation,Open or Active,Bayview,37.73678418944183,-122.38011105053184
13548,2020-02-01T15:30:00.000,15:30,2020,Saturday,Stolen Property,"Stolen Property, Possession with Knowledge, Re...",Cite or Arrest Adult,Taraval,37.7212992943916,-122.45212279953367


In [11]:
result_renamed.groupby(['Week', 'Category']).count()

Datetime  Time  Year  Description  Resolution  \
Week      Category                                                              
Friday    Arson                        6     6     6            6           6   
          Assault                    123   123   123          123         123   
          Burglary                   104   104   104          104         104   
          Case Closure                 4     4     4            4           4   
          Civil Sidewalks              2     2     2            2           2   
...                                  ...   ...   ...          ...         ...   
Wednesday Vehicle Impounded            4     4     4            4           4   
          Vehicle Misplaced            1     1     1            1           1   
          Warrant                     98    98    98           98          98   
          Weapons Carrying Etc        12    12    12           12          12   
          Weapons Offense             13    13    13           13          13   

                                Police_Dist  Lat  Lon  
Week      Category                                     
Friday    Arson                           6    6    6  
          Assault                       123  123  123  
          Burglary                      104  104  104  
          Case Closure                    4    4    4  
          Civil Sidewalks                 2    2    2  
...                                     ...  ...  ...  
Wednesday Vehicle Impounded               4    4    4  
          Vehicle Misplaced               1    1    1  
          Warrant                        98   98   98  
          Weapons Carrying Etc           12   12   12  
          Weapons Offense                13   13   13  

[276 rows x 8 columns]

In [25]:
result_sorted = result_renamed.sort_values(by=['Datetime']).reset_index()
result_sorted

,index,Datetime,Time,Year,Week,Category,Description,Resolution,Police_Dist,Lat,Lon
0,8693,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, From Locked Vehicle, >$950",Open or Active,Northern,37.774598784465354,-122.42588849947774
1,2449,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, Other Property, $50-$200",Open or Active,Park,37.77571358981224,-122.43965650508086
2,1555,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Drug Offense,Marijuana Offense,Open or Active,Ingleside,37.751911293506055,-122.42398839864852
3,6908,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, Other Property, >$950",Open or Active,Southern,37.78853592966637,-122.39603476680188
4,12750,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Non-Criminal,Found Property,Open or Active,Richmond,37.78068110285063,-122.4717754607559
...,...,...,...,...,...,...,...,...,...,...,...
12859,83,2020-02-08T19:43:00.000,19:43,2020,Saturday,Other Miscellaneous,Investigative Detention,Cite or Arrest Adult,Mission,37.76088893209152,-122.4350007026991
12860,271,2020-02-08T19:43:00.000,19:43,2020,Saturday,Other Offenses,"Alcohol, Consuming In Public View",Cite or Arrest Adult,Mission,37.76088893209152,-122.4350007026991
12861,207,2020-02-08T19:51:00.000,19:51,2020,Saturday,Malicious Mischief,"Malicious Mischief, Breaking Windows",Open or Active,Bayview,37.72688863114503,-122.39721349842785
12862,260,2020-02-08T19:51:00.000,19:51,2020,Saturday,Offences Against The Family And Children,Domestic Violence (secondary only),Open or Active,Bayview,37.72688863114503,-122.39721349842785


In [27]:
result_sorted.drop(['index'],axis=1)

,Datetime,Time,Year,Week,Category,Description,Resolution,Police_Dist,Lat,Lon
0,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, From Locked Vehicle, >$950",Open or Active,Northern,37.774598784465354,-122.42588849947774
1,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, Other Property, $50-$200",Open or Active,Park,37.77571358981224,-122.43965650508086
2,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Drug Offense,Marijuana Offense,Open or Active,Ingleside,37.751911293506055,-122.42398839864852
3,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Larceny Theft,"Theft, Other Property, >$950",Open or Active,Southern,37.78853592966637,-122.39603476680188
4,2020-01-01T00:00:00.000,00:00,2020,Wednesday,Non-Criminal,Found Property,Open or Active,Richmond,37.78068110285063,-122.4717754607559
...,...,...,...,...,...,...,...,...,...,...
12859,2020-02-08T19:43:00.000,19:43,2020,Saturday,Other Miscellaneous,Investigative Detention,Cite or Arrest Adult,Mission,37.76088893209152,-122.4350007026991
12860,2020-02-08T19:43:00.000,19:43,2020,Saturday,Other Offenses,"Alcohol, Consuming In Public View",Cite or Arrest Adult,Mission,37.76088893209152,-122.4350007026991
12861,2020-02-08T19:51:00.000,19:51,2020,Saturday,Malicious Mischief,"Malicious Mischief, Breaking Windows",Open or Active,Bayview,37.72688863114503,-122.39721349842785
12862,2020-02-08T19:51:00.000,19:51,2020,Saturday,Offences Against The Family And Children,Domestic Violence (secondary only),Open or Active,Bayview,37.72688863114503,-122.39721349842785


In [39]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
database = client['Police_reports']
collection = database['Incidents']

collection.insert_many(result_sorted.to_dict('records'))

In [60]:
# data = pd.DataFrame(list(collection.find({"Category": "Larceny Theft"})))
# data = pd.DataFrame(list(collection.find()))
# data = loads(dumps(collection.find()))
data = collection.find_one()
data

{'_id': ObjectId('5e41c0fa5a30f27f733bdb42'),
 'index': 8693,
 'Datetime': '2020-01-01T00:00:00.000',
 'Time': '00:00',
 'Year': '2020',
 'Week': 'Wednesday',
 'Category': 'Larceny Theft',
 'Description': 'Theft, From Locked Vehicle, >$950',
 'Resolution': 'Open or Active',
 'Police_Dist': 'Northern',
 'Lat': '37.774598784465354',
 'Lon': '-122.42588849947774'}